In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

Load the processed CSV file

In [18]:
input_file = './data/processed_data.csv'

In [19]:
df = pd.read_csv(input_file)

Remove rows with NaN values

In [20]:
df.dropna(inplace=True)

df['DateTime'] = pd.to_datetime(df['DateTime'], format="%Y%m%d%H%M")

Define the threshold for removing extreme Y values

In [21]:
threshold = 3.5  # Adjust this value based on your dataset and requirements

Calculate the mean and standard deviation of Y

In [22]:
mean_Y = np.mean(df['Temperature'])
std_Y = np.std(df['Temperature'])

Define the range of acceptable Y values

In [23]:
lower_bound = mean_Y - threshold * std_Y
upper_bound = mean_Y + threshold * std_Y

Filter out rows with Y values outside the acceptable range

In [24]:
df = df[(df['Temperature'] >= lower_bound) & (df['Temperature'] <= upper_bound)]

Extract the columns

In [25]:
X = df[['Month', 'Date', 'Time', 'Previous Day Average', 'Two Days Before Average', 'Three Days Before Average', 'Last 7 Days Average']].values
Y = df['Temperature'].values

Normalize X values

In [26]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

Split the data into training and testing sets

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Define the model architecture

In [28]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', input_shape=(7,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    # tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1) # Output layer with 1 unit for air temperature
])

In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# Compile the model
model.compile(optimizer=optimizer, loss="mean_absolute_error")

Define early stopping and learning rate scheduler

In [30]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

Train the model

In [31]:
history = model.fit(X_train, Y_train, epochs=500, batch_size=64, validation_data=(X_test, Y_test), callbacks=[early_stopping, lr_scheduler])

Epoch 1/500
1670/1670 [==============================] - 14s 7ms/step - loss: 11.2046 - val_loss: 7.8436 - lr: 0.0010
Epoch 2/500
1670/1670 [==============================] - 12s 7ms/step - loss: 8.5324 - val_loss: 7.4651 - lr: 0.0010
Epoch 3/500
1670/1670 [==============================] - 12s 7ms/step - loss: 7.7503 - val_loss: 6.7712 - lr: 0.0010
Epoch 4/500
1670/1670 [==============================] - 12s 7ms/step - loss: 5.9659 - val_loss: 4.6791 - lr: 0.0010
Epoch 5/500
1670/1670 [==============================] - 12s 7ms/step - loss: 5.3915 - val_loss: 4.7914 - lr: 0.0010
Epoch 6/500
1670/1670 [==============================] - 12s 7ms/step - loss: 5.1652 - val_loss: 4.9644 - lr: 0.0010
Epoch 7/500
1670/1670 [==============================] - 12s 7ms/step - loss: 5.0332 - val_loss: 4.5403 - lr: 0.0010
Epoch 8/500
1670/1670 [==============================] - 11s 7ms/step - loss: 4.9976 - val_loss: 4.7273 - lr: 0.0010
Epoch 9/500
1670/1670 [==============================] - 12s 7m

Evaluate the model

In [32]:
loss = model.evaluate(X_test, Y_test)
print("Test loss:", loss)

835/835 [==============================] - 1s 1ms/step - loss: 2.8399
Test loss: 2.839933395385742


Make predictions

In [33]:
predictions = model.predict(X_test)

835/835 [==============================] - 1s 904us/step


Calculate accuracy and mean squared error (MSE)

In [34]:
from sklearn.metrics import mean_absolute_error
accuracy = 100 - np.mean(np.abs((Y_test - predictions) / Y_test)) * 100
mse = mean_squared_error(Y_test, predictions)
mas = mean_absolute_error(Y_test, predictions)

: 

: 

In [ ]:
print("Accuracy:", accuracy)
print("Mean Squared Error (MSE):", mse)

Plot error (predicted - actual)

In [ ]:
error = predictions.flatten() - Y_test

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(Y_test, error)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Actual Air Temperature')
plt.ylabel('Error (Predicted - Actual)')
plt.title('Error Plot')
plt.show()

Plot training and validation loss over epochs

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

Print some example predictions

In [ ]:
for i in range(10):
    print("Predicted:", predictions[i])
    print("Actual:", Y_test[i])
    print()

Save the TensorFlow model

In [ ]:
model.save("./model.keras")